# Priority Places for Food - Data Preparation

# Set-up
## Install required packages

In [ ]:
!pip install geovoronoi
!pip install openpyxl
!pip install odfpy

In [ ]:
import urllib3
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, LineString, Point, shape
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import operator
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'DistanceCalculator'))
from bipartite_nearest_distance_calculator import calculate_bipartite_nearest_distance, count_bipartite_within_tolerance
from urllib.request import urlopen
from shutil import copyfileobj, unpack_archive
import openpyxl

## Set data directories

In [ ]:
raw_data_directory = '/workspaces/priority-places-calculator/data/raw/'
processed_data_directory = '/workspaces/priority-places-calculator/data/processed/'

## Proximity to supermarket retail facilities

### Average distance to nearest large grocery store

For England, Scotland and Wales, indicator data obtained from e-food desert index of Newing et al., 2020. Location data of large grocery stores obtained from Geolytix Retails Points v15 and distances computed using GIS software. Code for Northern Ireland calculation shown below.

In [ ]:
if not os.path.exists(raw_data_directory + 'geolytix/Previous Versions/geolytix_retailpoints_v15_202001.csv'):
    # Note the format for the direct download link.
    url = 'https://drive.google.com/uc?id=1B8M7m86rQg2sx2TsHhFa2d-x-dZ1DbSy'
    with urlopen(url) as in_stream, open(raw_data_directory + 'geolytix.zip', 'wb') as out_file:
        copyfileobj(in_stream, out_file)
    unpack_archive(raw_data_directory + 'geolytix.zip', raw_data_directory + 'geolytix')

supermarkets = pd.read_csv(raw_data_directory + 'geolytix/Previous Versions/geolytix_retailpoints_v15_202001.csv')
supermarkets = gpd.GeoDataFrame(supermarkets, geometry=gpd.points_from_xy(supermarkets['long_wgs'], supermarkets['lat_wgs']), crs=4326)
large_supermarkets = supermarkets[supermarkets.size_band.isin(['15,069 < 30,138 ft2 (1,400 < 2,800 m2)', '30,138 ft2 > (2,800 m2)'])]
large_supermarkets = gpd.GeoDataFrame(large_supermarkets, geometry=gpd.points_from_xy(large_supermarkets['long_wgs'], large_supermarkets['lat_wgs']), crs=4326)

In [ ]:
country_outline = gpd.read_file('https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Countries_December_2021_UK_BGC_2022/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')
country_outline = country_outline.to_crs('EPSG:4326')
country_outline = country_outline.dissolve()

In [ ]:
large_supermarkets = gpd.sjoin(large_supermarkets, country_outline, how="inner", predicate='intersects')

In [ ]:
def load_postcode_data():
    if not os.path.exists(raw_data_directory + 'postcodes.csv'):
        postcodes = pd.read_csv('https://geoportal.statistics.gov.uk/datasets/0d58125f0f9f4c81b6370a10f5ea3309_0.csv')
        postcodes.to_csv(raw_data_directory + 'postcodes.csv', index=False)
    else:
        postcodes = pd.read_csv(raw_data_directory + 'postcodes.csv')
        postcodes = gpd.GeoDataFrame(postcodes, geometry=gpd.points_from_xy(postcodes['X'], postcodes['Y']), crs=4326)
        print(postcodes.head())
    return postcodes#[['PCD', 'LSOA11', 'OSLAUA', 'X', 'Y', 'geometry']]

postcodes = load_postcode_data()

In [ ]:
lookup = pd.read_csv(' +  'PCD_OA_LSOA_MSOA_LAD_MAY22_UK_LU.csv', encoding='latin-1')
lookup_2 =pd.read_csv(data_directory +  'Local_Authority_District_to_County_(April_2021)_Lookup_in_England.csv')

lookup_eng =lookup.loc[lookup['lsoa11cd'].str.startswith('E')==True,:]


lookup_eng['cleaned_ladnm']=lookup_eng['ladnm']

l =['East ','North East ','North ','North West ',
              'West ','South West ','South ','South East ']
lookup_eng['cleaned_ladnm'] =lookup_eng['cleaned_ladnm'].str.replace('|'.join(l), '', regex=True).str.strip()


lookup_eng=lookup_eng.merge(lookup_2[['LAD21NM','CTY21NM']], left_on ='ladnm', right_on='LAD21NM')

In [ ]:
pntsB = large_supermarkets[['id', 'geometry']].to_crs(4326)
pntsB.set_index('id', inplace=True)

pntsA = postcodes[['PCD', 'geometry']].to_crs(4326)
pntsA.set_index('PCD', inplace=True)

Polygon = country_outline.envelope.values[0]

In [ ]:
dist = calculate_bipartite_nearest_distance(pntsA, pntsB, Polygon)
dist.to_csv(processed_data_directory + 'postcode_to_nearest_large_supermarket_distance.csv', index=True)

### Average count of stores within 1km

For England, Scotland and Wales, indicator data obtained from e-food desert index of Newing et al., 2020. Location data of large grocery stores obtained from Geolytix Retails Points v15 and distances computed using GIS software. Code for Northern Ireland shown below. 

In [ ]:
tolerance = 1000
pntsA = postcodes[['PCD', 'geometry']]
supermarket_within_1km = count_bipartite_within_tolerance(pntsA, pntsB, 1000)
supermarket_within_1km.to_csv(processed_data_directory + 'postcode_to_nearest_large_supermarket_1km_count.csv', index=True)

## Accessibility to supermarket retail facilities

### Average travel distance (based on a custom built spatial interaction model)

For England, Scotland and Wales, values are taken from the same indicator in the e-food desert index of Newing et al., 2020. 

In [ ]:
if not os.path.exists(raw_data_directory + 'EFDI_IndividualVariables.csv'):
    # TODO: How to source this file.
    raise NotImplementedError()

efdi_variables = pd.read_csv(raw_data_directory + 'EFDI_IndividualVariables.csv')
average_travel_distance = efdi_variables[['id', 'Average trip distance']]
average_travel_distance.set_index('id', inplace=True)
average_travel_distance.to_csv(processed_data_directory + 'average_trip_distance.csv', index=True)

### Accessibility via public transport (Govt Journey Time Statistics 2017 - 2020)

For England, Scotland and Wales, values are taken from the same indicator in the e-food desert index of Newing et al., 2020. Data is based on ...

In [ ]:
if not os.path.exists(raw_data_directory + 'EFDI_IndividualVariables.csv'):
    # TODO: How to source this file.
    raise NotImplementedError()

efdi_variables = pd.read_csv(raw_data_directory + 'EFDI_IndividualVariables.csv')
public_transport_retail = efdi_variables[['id', 'pt-retail']]
public_transport_retail.set_index('id', inplace=True)
public_transport_retail.to_csv(processed_data_directory + 'public_transport_retail.csv', index=True)

## Access to online deliveries

### Online groceries availability (Newing et al, 2020). E,S,W

In [ ]:
if not os.path.exists(raw_data_directory + 'EFDI_IndividualVariables.csv'):
    # TODO: How to source this file.
    raise NotImplementedError()

efdi_variables = pd.read_csv(raw_data_directory + 'EFDI_IndividualVariables.csv')
online_groceries_availability = efdi_variables[['id', 'totaldeliv']]
online_groceries_availability.set_index('id', inplace=True)
online_groceries_availability.to_csv(processed_data_directory + 'online_groceries_availability.csv', index=True)

### Propensity to shop online (CDRC Internet User Classification 2018). E,S,W.

In [ ]:
if not os.path.exists(raw_data_directory + 'EFDI_IndividualVariables.csv'):
    # TODO: How to source this file.
    raise NotImplementedError()

efdi_variables = pd.read_csv(raw_data_directory + 'EFDI_IndividualVariables.csv')
online_shop_propensity = efdi_variables[['id', 'zshoponline']]
online_shop_propensity.set_index('id', inplace=True)
online_shop_propensity.to_csv(processed_data_directory + 'online_shop_propensity.csv', index=True)

## Proximity to non-supermarket food provision

In [ ]:
# This cell cleans and processes the FSA data
def load_fsa_data(postcodes):
    
    if not os.path.exists(raw_data_directory + 'FSA_establishment_data.csv'):
        # TODO: How to source this file
        raise NotImplementedError()

    fsa = pd.read_csv(raw_data_directory + 'FSA_establishment_data.csv')

    # Relating to 'Proximity to and density of Non-supermarket food provision', restrict the data to 'Retailers - other'
    fsa = fsa[fsa['BusinessType'].isin(['Retailers - other'])]

    # The largest proportion of geographic data is avialable at the postcode level. 
    # In an effort to clean up the location data, let's take any data with a missing long/lat field and see if we can map the postcode to lon/lat positions.
    print('Total records: %d' % fsa.shape[0])
    print('Total records without a geocode: %d' % fsa['Geocode.Latitude'].isna().sum())

    joined = fsa.merge(postcodes[['PCD', 'X', 'Y', 'geometry']], left_on='PostCode', right_on='PCD', how='left')
    joined['Geocode.Latitude'] = joined['Geocode.Latitude'].fillna(joined.Y)
    joined['Geocode.Longitude'] = joined['Geocode.Longitude'].fillna(joined.X)
    fsa = joined[fsa.columns]

    print('Total records without a geocode: %d' % fsa['Geocode.Latitude'].isna().sum())

    # Filter out records without a geocode
    fsa = fsa[~fsa['Geocode.Latitude'].isna()]

    fsa = gpd.GeoDataFrame(fsa, geometry=gpd.points_from_xy(fsa['Geocode.Longitude'], fsa['Geocode.Latitude']), crs='EPSG:4326')

    return fsa
    
# Load the postcode data if not already loaded                      
if 'postcodes' not in globals():
    postcodes = load_postcode_data()

fsa = load_fsa_data(postcodes)




### Distance to nearest non-supermarket retail food store (Food Standards Agency, accessed 2022-08-23). E,S,W,NI

In [ ]:
if 'postcodes' in globals() and 'fsa' in globals():
    for region in postcodes[~postcodes.OSLAUA.isna()].OSLAUA.str[0:3].unique():
        pcds_rgn = postcodes[postcodes['OSLAUA'].str[0:3] == region]
        pntsA = pcds_rgn[['PCD', 'geometry']]
        pntsA.set_index('PCD', inplace=True)
        pntsB = fsa[['FHRSID', 'geometry']]
        polygon = country_outline.envelope.values[0]
        dist = calculate_bipartite_nearest_distance(pntsA, pntsB, polygon)
        dist.to_csv(processed_data_directory + region + '_pcd_nonsupermarket_dist.csv')
else:
    raise RuntimeError('postcode and fsa objects not defined')

### Count of non-supermarket retail food stores within 1km (Food Standards Agency, accessed 2022-08-23). E,S,W,NI

In [ ]:
def calculate_nonsupermarket_1kmcount():
    # Check that postcodes and fsa have been defined before running the function
    if 'postcodes' in globals() and 'fsa' in globals():
        
        # Convert FSA data to UK BNG
        pntsB = fsa[['FHRSID', 'geometry']].to_crs(27700)
        
        # Loop through regions to batch things up a bit
        for region in postcodes[~postcodes.OSLAUA.isna()].OSLAUA.str[0:3].unique():
            
            print('Region currently processing: %s' % region)
            # Initialise regional count
            pcd_intersect_counts = pd.DataFrame()
            
            # Filter on region 
            pcds_rgn = postcodes[postcodes['OSLAUA'].str[0:3] == region]
            
            # Reproject to UK BNG
            pcds_rgn = pcds_rgn.to_crs(27700)
            
            # Calculate the postcode buffer
            pcds_rgn.geometry = pcds_rgn.buffer(1000)
            
            print('Number of PCDs in region: %d' % (pcds_rgn.shape[0]))
            
            # Batch the analysis
            batchsize = int(pcds_rgn.shape[0] / 10.0)
            row = 0
            batch_counter = 0
            while row < pcds_rgn.shape[0]:
                print("Batch %d -- Processing pcds_rgn rows %d to %d" % (batch_counter, row, row+batchsize))
                
                # Join and group by in a single step to prevent any large dfs hanging around
                pcd_intersect_counts_batch = gpd.sjoin(pcds_rgn[row:row+batchsize], pntsB, how='left').groupby('PCD')['FHRSID'].count()
                
                # Save the batch
                pcd_intersect_counts = pd.concat([pcd_intersect_counts, pcd_intersect_counts_batch])
                batch_counter += 1
                row += batchsize
                
            pcd_intersect_counts.to_csv(processed_data_directory + region + '_pcd_nonsupermarket_1kmcount.csv')
    else:
        raise RuntimeError('postcode and fsa objects not defined')
        
calculate_nonsupermarket_1kmcount()

### Average distance to nearest market (CDRC data from National Market Traders Federation 2016-2019). E,W.

In [ ]:
# Data can be downloaded from https://data.cdrc.ac.uk/dataset/national-market-traders-federation
# and should be saved in the raw_data directory.

if not os.path.exists(raw_data_directory + 'nmftmarkets.xlsx'):
    raise FileNotFoundError("National market traders federation data not found in raw data directory. Please download from: https://data.cdrc.ac.uk/dataset/national-market-traders-federation")

nmftmarkets = pd.read_excel(raw_data_directory + 'nmftmarkets.xlsx')
nmftmarkets = gpd.GeoDataFrame(nmftmarkets, geometry=gpd.points_from_xy(nmftmarkets.longitude, nmftmarkets.latitude), crs=4326)
nmftmarkets = nmftmarkets[nmftmarkets['type_specific_mkv'].isin(['Retail', 'Food and Drink', 'Food and Drink/Arts and Crafts', 'General', 'Speciality Food and Drink', 'Food'])]

if 'postcodes' not in globals():
    postcodes=load_postcode_data()

pntsA = postcodes[['PCD', 'geometry']]
pntsA.set_index('PCD', inplace=True)
pntsB = nmftmarkets[['name_mkv', 'geometry']]
polygon = country_outline.envelope.values[0]
dist = calculate_bipartite_nearest_distance(pntsA, pntsB, polygon)
dist.to_csv(processed_data_directory + 'pcd_nmftmarkets_dist.csv')

### Average count of markets within 1km (CDRC data from National Market Traders Federation 2016-2019). E,W.

In [ ]:
market_within_1km = count_bipartite_within_tolerance(pntsA, pntsB, 1000)
market_within_1km.to_csv(processed_data_directory + 'postcode_market_1km_count.csv', index=True)

## Socio-demographic barriers

### Proportion of population experiencing income deprivation (UK Govt Index of Multiple Deprivation 2017-2020). E,S,W,NI. 

In [ ]:
# Download for different deprivation indicators...



### Proportion of population with no car access (UK Census 2011). E,S,W,NI.

In [ ]:
# Census data download...

## Need for family food support

### Free school meal eligibility. E,S,W,NI.

- state-funded nursery schools
- state-funded primary schools
- state-funded secondary schools
- special schools, including state-funded special schools and non-maintained special schools
- pupil referral units, including free school alternative provision and academy alternative provision

https://explore-education-statistics.service.gov.uk/data-catalogue/free-school-meals-autumn-term/2020-21-autumn-term

In [ ]:
url = 'https://content.explore-education-statistics.service.gov.uk/api/releases/df258c3a-5be2-4b6c-9f20-08d88fd210c7/files'

if not os.path.exists(raw_data_directory + 'fsm_eng/data/fsm_new_starts_autumn20.csv'):
    with urlopen(url) as in_stream, open(raw_data_directory + 'fsm_eng.zip', 'wb') as out_file:
        copyfileobj(in_stream, out_file)
        unpack_archive(raw_data_directory + 'fsm_eng.zip', raw_data_directory + 'fsm_eng')

fsm_eng = pd.read_csv(raw_data_directory + 'fsm_eng/data/fsm_new_starts_autumn20.csv')
fsm_eng = fsm_eng.loc[fsm_eng['time_period']==202021,:]
fsm_eng = fsm_eng[(fsm_eng['phase']=='Total') & (fsm_eng['geographic_level']=='Local authority')]

In [ ]:
lsoa11_oslaua_lookup = postcodes.groupby('LSOA11')['OSLAUA'].first()


test = fsm_eng.merge(lsoa11_oslaua_lookup, left_on ='new_la_code', right_on='OSLAUA', how='right', indicator=True)
test['_merge'].value_counts()

test[test['_merge']=='both']['OSLAUA'].nunique()

In [ ]:
# Wales
url = "https://www.gov.wales/sites/default/files/statistics-and-research/2022-08/schools-census-results-as-at-february-2022-tables-227.ods"
with urlopen(url) as in_stream, open(raw_data_directory + "schools-census-results-as-at-february-2022-tables-227.ods", 'wb') as out_file:
    copyfileobj(in_stream, out_file)

xls = pd.ExcelFile(raw_data_directory + 'schools-census-results-as-at-february-2022-tables-227.ods')
fsm_wal = pd.read_excel(xls, 13, skiprows=2)
worksheet_title = pd.read_excel(xls,13,nrows=1)
print(worksheet_title)

fsm_wal = fsm_wal[['Local authority', 'All pupils - total - per cent']].dropna()    

In [ ]:


utla_lookup = gpd.read_file("https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/LTLA21_UTLA21_EW_LU_9bbac05558b74a88bda913ad5bf66917/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson")

In [ ]:
'https://www.arcgis.com/sharing/rest/content/items/3e265c6a114f425fbd92e863977e698a/data'

In [ ]:
fsm_lookup_eng = postcodes[postcodes['OSLAUA'].str[0]=='E'].merge(utla_lookup, left_on='OSLAUA', right_on='LTLA21CD', how='left', indicator=True)

In [ ]:

bespoke_lookup = fsm_lookup_eng[['LSOA11', 'UTLA21CD']].drop_duplicates()
oldutla_lookup = gpd.read_file('https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/LSOA11_UTLA17_EW_LU_fc16626637fa480f8164e11378f5a1e4/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')
oldutla_lookup.rename({'LSOA11CD':'LSOA11'}, inplace=True, axis=1)
bespoke_lookup = bespoke_lookup.merge(oldutla_lookup[['LSOA11', 'UTLA17CD']], left_on='LSOA11', right_on='LSOA11', how='left')
lad2011lookup = gpd.read_file('https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/LSOA01_LSOA11_LAD11_EW_LU_ddfe1cd1c2784c9b991cded95bc915a9/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')
lad2011lookup.rename({'LSOA11CD':'LSOA11'}, inplace=True, axis=1)
lad2011lookup = lad2011lookup[lad2011lookup['LAD11NM'].isin(['Gateshead', 'Northumberland'])]
bespoke_lookup = bespoke_lookup.merge(lad2011lookup[['LSOA11', 'LAD11CD']], left_on='LSOA11', right_on='LSOA11', how='left')
bespoke_lookup['lookup_cd'] = bespoke_lookup['LAD11CD'].fillna(bespoke_lookup['UTLA17CD']).fillna(bespoke_lookup['UTLA21CD'])


fsm_eng_lsoa = fsm_eng.merge(bespoke_lookup, right_on='lookup_cd', left_on ='new_la_code', how='left', indicator=True)
fsm_eng_lsoa.rename({'_merge': 'merge2'}, inplace=True, axis=1)

In [ ]:
fsm_eng_lsoa.merge2.value_counts()

### Healthy start voucher usage (England and Wales only). E,W.

In [ ]:
# LA level
HS_uptake = pd.read_csv(data_directory + 'Healthy_start_uptake_March_2022.csv')
# Remove 'Teaching' from several local authority names in Wales
HS_uptake['Local Authority'] = HS_uptake['Local Authority'].astype(str).str.replace(' Teaching',"")
# Change local authority wording to match lookup 
HS_uptake['Local Authority'] = HS_uptake['Local Authority'].astype(str).str.replace("Taff","Taf")
HS_uptake['Local Authority'] = HS_uptake['Local Authority'].astype(str).str.replace('Anglesey','Isle of Anglesey')
HS_uptake['Local Authority'] = HS_uptake['Local Authority'].astype(str).str.replace('Kings Lynn and West Norfolk',"King's Lynn and West Norfolk")

lookup = pd.read_csv(data_directory + 'PCD_OA_LSOA_MSOA_LAD_AUG21_UK_LU.csv', encoding='Latin-1')
census_21 =pd.read_csv(data_directory + 'popualtion_change_2011_2021_fig_3.csv')
# Census population data (Local Authority -Child Population)
census_21_child= pd.read_csv(data_directory + 'Local_Authority_pop_child.csv')

# join look up table to census poualtion (2021) for all and child popualtions
lookup =lookup.merge(census_21[['LA code', 'Usual resident population, 2011',
       'Usual resident population, 2021', 'Percentage change']], left_on='ladcd', right_on='LA code', how='left')
lookup =lookup.merge(census_21_child, left_on='ladcd', right_on='Area code', how='left')

# join healthy start to lcoal authority in look up table
HS_uptake =lookup.merge(HS_uptake, left_on=['ladnm'], right_on ='Local Authority', how='left', indicator=True)

HS_uptake_lsoa = HS_uptake.groupby('lsoa11cd').agg({'ladnm':'first','Country':'first','Local Authority':'first',
                                                   'Total Entitled Beneficiaries':'first','Total Eligible Beneficiaries':'first',
                                                   'Usual resident population, 2021':'first', 
                                                   'Uptake (%)':'first', 'Aged 4 years and under':'first',
                                                   'Aged 5 to 9 years':'first', 'Aged 10 to 14 years':'first', 
                                                   'Aged 15 to 19 years':'first', '_merge':'first'})

HS_uptake_lsoa['Percentage_Population_Eligible_HS'] =HS_uptake_lsoa['Total Eligible Beneficiaries']/HS_uptake_lsoa['Usual resident population, 2021']*100
HS_uptake_lsoa['Percentage_under_5_Population_Eligible_HS'] =HS_uptake_lsoa['Total Eligible Beneficiaries']/HS_uptake_lsoa['Aged 4 years and under']*100
HS_uptake_lsoa['Percentage_Population_Using_HS'] =HS_uptake_lsoa['Total Entitled Beneficiaries']/HS_uptake_lsoa['Usual resident population, 2021']*100
HS_uptake_lsoa['Percentage_under_5_Population_Using_HS'] =HS_uptake_lsoa['Total Entitled Beneficiaries']/HS_uptake_lsoa['Aged 4 years and under']*100

### Distance to nearest food bank (https://www.givefood.org.uk/, accessed 2022-08-19). E,S,W,NI. 

In [ ]:
foodbanks_df = pd.read_json(data_directory + 'foodbanks_givefood_api_19Aug22.json')
foodbanks_df['lat'] = pd.to_numeric(foodbanks_df.loc[:, 'lat_lng'].str.split(',').apply(operator.itemgetter(0)))
foodbanks_df['lng'] = pd.to_numeric(foodbanks_df.loc[:, 'lat_lng'].str.split(',').apply(operator.itemgetter(1)))
foodbanks_df = gpd.GeoDataFrame(foodbanks_df, 
                                geometry=gpd.points_from_xy(foodbanks_df['lng'], foodbanks_df['lat']),
                                crs='EPSG:4326')

country_outline = gpd.read_file(data_directory + 'Countries_(December_2021)_UK_BFC.zip')
country_outline = country_outline.to_crs('EPSG:4326')
country_outline = country_outline.dissolve()

postcodes = pd.read_csv(data_directory + 'ONS_Postcode_Directory_(Latest)_Centroids.csv')
postcodes = gpd.GeoDataFrame(postcodes, 
                             geometry=gpd.points_from_xy(postcodes['long'], postcodes['lat']), 
                             crs='EPSG:4326')

pntsB = foodbanks_df[['slug', 'geometry']]
pntsB.set_index('slug', inplace=True)

pntsA = postcodes[['pcd', 'geometry']]
pntsA.set_index('pcd', inplace=True)

Polygon = country_outline.envelope.values[0]

dist = calculate_bipartite_nearest_distance(pntsA, pntsB, Polygon)
dist.to_csv(data_directory + 'postcode_to_nearest_foodbank_distance.csv', index=True)

print('Time taken: %s' % (time.time() - t1))

## Fuel Poverty

### Proportion of households in fuel poverty (2017 - 2020). E,S,W.

Low Income Low Energy Efficiency (LILEE) fuel poverty metric (2020 data) - England https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1056802/fuel-poverty-methodology-handbook-2022-lilee-with-projection.pdf

https://www.gov.uk/government/statistics/sub-regional-fuel-poverty-data-2022


Fuel poverty calculation (England)- estimates the number and proportion of fuel poor households for smaller geographical areas and measured using proxy measures available for low level geographic areas and constrained to the national data drawn from the English Housing Survey (EHS)


Application of the sub-regional model requires specific local area data for a variety of demographic and socio-economic factors, which are derived from the 2011 Census small area datasets

Ideally, the EHS data would be used directly to model at the local level. Unfortunately, this is not possible because of the relatively small sample used for the survey, which does not give sufficient coverage for each of the 309 Local Authorities in England or the individual Census Output Areas (COAs) 

To allow results to be produced at this level, we use the EHS data to create a model which can then be applied to a national (household level) dataset. The limitation of this is the possibility that small areas which are atypical in condition are unlikely to be identified by the model. It is therefore essential, wherever possible, to compare the modelled results to local data.


Model:
1. Regression model testing: 
- A large number of independent variables are individually tested against the binary fuel poverty indicator to determine which have the highest coefficient of determination, i.e. which are most sensitive in predicting the fuel poverty outcome.
- Variables are then grouped according to this coefficient and an iterative selection is made by running logistic regression on a developing group of these variables, with those least likely to contribute to final model predictions being dropped at each stage
- The final set of variables contributing to the model varies each year depending on the results of the regression testing.


The 2020 LILEE sub-regional fuel poverty model is based on the following seven variables, derived from public and commercial sources:
- Tenure – owner occupied, private rented, social rented (Experian)
- Dwelling age – pre-1919, 1919 to 1944, 1945 to 1980, post-1980 (Experian)
- Household composition (Experian)
- Mosaic household classifications by postcode – 15 socio-economic groups as detailed in the Mosaic guide (Experian)
- Dwelling type – terraced, detached, semi-detached, bungalow, flat (Experian)
- Government office region (Experian)
- Employment status - proportion of households in COA with household member in full time employment (2011 Census)

2. Model application
- The coefficients for each category of the final model’s independent variables are run to give the predicted outcome for each household in each area. 
- These are then combined to produce the total numbers and proportion of fuel poor at COA level.

3. Out put area cloning (addressing missing data):
- After this process, a small number of COAs have missing fuel poverty data. 
- These gaps are filled by identifying COAs with complete data, which share identical or very similar characteristics to the missing cases. 
- Census data is used for this comparison as it records proportions of particular characteristic rather than discrete categories, and so the process can more accurately match pairs of COAs. The Census variables used in this matching process were those found in the final regression model, or which were omitted at a very late stage of testing. 

4. Consolidation to national figures:
- In order to provide an exact match between the modelled household totals and the national figures (for all households and fuel poor households), a final adjustment is made in which small incremental changes are made randomly at a COA level, until the totals of all households and fuel poor households match the published regional totals

5. The adjusted figures are re-checked against the published totals before the data is aggregated to LSOA, Local Authority, Parliamentary Constituency and Region level.


Welsh equivalent of fuel poverty data: https://gov.wales/sites/default/files/statistics-and-research/2020-03/welsh-housing-conditions-survey-whcs-2017-18-local-area-fuel-poverty-estimates-modelling-and-results-summary-071.pdf (taken from pdf)


Scottish equivalent of fuel poverty data: https://statistics.gov.scot/resource?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Ffuel-poverty-shcs (local authority not data zone) 
A household is considered to be in fuel poverty if, in order to maintain a satisfactory heating regime:
the total fuel costs necessary for the home are more than 10% of the household’s adjusted net income (after housing costs), and
if after deducting fuel costs, benefits received for a care need or disability and childcare costs, the household’s remaining adjusted net income is insufficient to maintain an acceptable standard of living. The remaining adjusted net income must be at least 90% of the UK Minimum Income Standard to be considered an acceptable standard of living, with an additional amount added for households in remote rural, remote small town and island areas.
Extreme fuel poverty follows the same definition except that a household would have to spend more than 20% of its adjusted net income (after hou
sing costs) on total fuel costs and maintain a satisfactory heating regime.

In [ ]:
fuel_eng = pd.read_csv(data_directory + 'Fuel_poverty_2020_England.csv')
fuel_wal = pd.read_csv(data_directory + 'Fuel_poverty_2018_Wales.csv')
fuel_scot = pd.read_csv(data_directory + 'Fuel_poverty_2017_19_Scotland.csv')

lookup = pd.read_csv(data_directory + 'PCD_OA_LSOA_MSOA_LAD_AUG21_UK_LU.csv', encoding='Latin-1')
census_21 =pd.read_csv(data_directory + 'popualtion_change_2011_2021_fig_3.csv')
lookup =lookup.merge(census_21[['LA code', 'Usual resident population, 2011','Usual resident population, 2021', 'Percentage change']], left_on='ladcd', right_on='LA code', how='left')

fuel_wal['Local_authority'] =fuel_wal['Local_authority'].astype(str).str.replace("Taff","Taf")
fuel_wal['Local_authority'] =fuel_wal['Local_authority'].astype(str).str.replace("The Vale","Vale")

# Join to look-up 
fuel_wal = lookup.merge(fuel_wal,  left_on='ladnm', right_on ='Local_authority', how='left', indicator=True)

# Remove areas with no match i.e. not in Wales
fuel_wal=fuel_wal.dropna(subset=['Percentage of households in fuel poverty '])

# Select columns to join to wider fuel poverty df
fuel_wales_to_join =fuel_wal[['lsoa11cd','Percentage of households in fuel poverty ','lsoa11nm']]
# Rename for consistent formatting 
fuel_wales_to_join.columns =['lsoa11cd','Percent of households in fuel poverty','lsoa11nm']

# Join to look-up 
fuel_scot =lookup.merge(fuel_scot, left_on='ladcd', right_on ='FeatureCode', how='left', indicator=True)

# Remove areas with no match i.e. not in Scotland
fuel_scot =fuel_scot.dropna(subset=['Percent of households in fuel poverty'])

# Select columns to join to wider fuel poverty df
fuel_scot_to_join =fuel_scot[['lsoa11cd','Percent of households in fuel poverty','lsoa11nm']]

# Select columns to join to wider fuel poverty df
fuel_eng_to_join = fuel_eng[['LSOA Code','Proportion of households fuel poor (%)','LSOA Name']]
# Rename for consistent formatting 
fuel_eng_to_join.columns =['lsoa11cd','Percent of households in fuel poverty','lsoa11nm']

fuel =pd.concat([fuel_scot_to_join,fuel_wales_to_join,fuel_eng_to_join])

fuel = fuel.drop_duplicates()
fuel.to_csv(data_directory + 'Fuel_poverty.csv', index=False)

### Prepayment meter prevalence, 2017. E,S,W.

https://www.gov.uk/government/statistics/electric-prepayment-meter-statistics

In [ ]:
prepayment = pd.read_csv(data_directory + 'Prepayment_meter_2017.csv')

households_lookup = pd.read_csv(data_directory + 'postcodes/postcodes_data.csv', usecols=['PCD', 'Occupied_Households','LSOA_DZ_Code'])

households_lookup = households_lookup.groupby('LSOA_DZ_Code')['Occupied_Households'].sum()

prepayment = prepayment.merge(households_lookup, left_on='Lower Layer Super Output Area (LSOA) Code', right_index=True, how='left', indicator=True)